In [1]:
import sys
sys.path.append("d:\\code\\demo\\smp\\GuaranteedDelivery")
import warnings
warnings.filterwarnings('ignore')

In [2]:
from core.models.chatglm2.jina_client import encode
from core.prompt import intent_recognition_prompt
from core.prompt import entity_recognition_prompt

import json

In [3]:
questions = []
with open('D:\\code\\demo\\smp\\GuaranteedDelivery\\data\\chatglm_llm_fintech_raw_dataset\\test_questions.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        questions.append(data['question'])

In [4]:
prompt = intent_recognition_prompt("博云新材在2020年对联营企业和合营企业的投资收益是多少元？")

In [5]:
response = encode(prompt, history=[])

In [6]:
with open("../data/test_temp/intent.txt", "w") as f:
    for i, question in enumerate(questions):
        f.write(f"Q[{i}]:{question}\n")
        prompt = intent_recognition_prompt(question)
        response = encode(prompt, history=[])
        f.write(f"R1:{response[0].text}\n")
        prompt = entity_recognition_prompt(question)
        response = encode(prompt, history=[])
        f.write(f"R2:{response[0].text}\n")
        if i > 10:
            break